### This generates new songs based off of the guitar licks that were intaken

In [1]:
%matplotlib inline

import mido
from mido import Message, MidiFile, MidiTrack
import numpy as np
from math import ceil
import keras
import random
import music21
import glob
import os
import json

from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
from keras.optimizers import RMSprop
import numpy

import pickle

Using TensorFlow backend.


### Load from disk

In [3]:
ls data/models/

test.h5             test.json           transposed_licks.p


In [4]:
model_dir = 'data/models/'
model_name = 'test'


# load json and create model
with open(model_dir + model_name + '.json', 'r') as json_file:
    loaded_model_json = json_file.read()

model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights(model_dir + model_name + '.h5')
print("Loaded model from disk")
 
# evaluate loaded model on test data
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

Loaded model from disk


### Load a sample passage

In [9]:
data_fname = 'data/models/transposed_licks.p'
with open(data_fname) as f:
    data = pickle.load(f)
    
X = data['X']
y = data['y']

In [27]:
ix = random.randint(0, X.shape[0])
psg_seed = X[ix,:,:]
psg_seed = psg_seed[None,:]

### s

In [31]:
def new_song_mat(model, psg_seed, tsteps=100):
    """ turns a passage seed to a song with #tsteps
    """
#     start_ix = random.randint(0, steps-maxlen - 1)
#     psg_seed = music_mat[start_ix:start_ix+maxlen,:]
#     x = psg_seed[None,:,:]

    new_song = []
    x = psg_seed
    new_song.append(x)
    nnotes = psg_seed.shape[-1]
    
    for _ in range(tsteps):

        preds = model.predict(x, verbose=0)[0]

        # this is a timestep slice
        new_note = np.zeros(nnotes)
        new_note[preds.argmax()] = 1
        new_note = new_note[None, None,:]

        # redo the seed psg
        psg = np.concatenate([x,new_note], axis=1)
        x = psg[:, 1:,:]

        new_song.append(new_note)

    new_song = np.concatenate(new_song, axis=1)
    new_song = np.squeeze(new_song)
    return new_song

In [43]:
def notelist_to_track(new_song, tick_step=100, note_range=[36,84]):   
    """ Converts a notelist to a miditrack.
    Assumes that adjacent notes are held
    """
    track = MidiTrack()
    nsteps = new_song.shape[0]
    last_event = 0
    active_notes = np.zeros(new_song.shape[-1])
    
    for ix in range(nsteps):
        step_slice = new_song[ix,:]

        diff = [ix for ix,(x,y) in enumerate(zip(step_slice, active_notes)) if x!=y]
        if len(diff) == 0:
            last_event += 1
            continue

        # this means there is a difference
        for note_ix in diff:
            # off to on
            note = note_ix+min(note_range)
            if active_notes[note_ix] == 0:
                track.append(Message('note_on', note=note, velocity=127, time=int(tick_step*last_event) ))
                active_notes[note_ix] = 1
            else:
                track.append(Message('note_off', note=note, velocity=127, time=int(tick_step*last_event)))
                active_notes[note_ix] = 0
            last_event = 0
    return track

In [32]:
mat = new_song_mat(model, psg_seed)

In [44]:
track = notelist_to_track(mat)

In [ ]:
mid = MidiFile()
mid.tracks.append(track)
